## Applied Data Science Capstone Week 3 Part 1

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

### Web Scraping Wikipedia Table using BeautifulSoup and Python  
reference: https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

In [2]:
page = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read().decode()
soup = BeautifulSoup(page, 'html.parser')

In [3]:
table = soup.find('table', class_='wikitable')
rows = table.find_all('tr')

In [4]:
data = []
for row in rows:
    data.append(row.text.split('\n')[1:-1])

In [5]:
df = pd.DataFrame(data[1:], columns = data[0])
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
df1 = df[df.Borough != 'Not assigned']
df1.reset_index(drop=True, inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Group by postcode and borough to remove duplicates, at the same time join neighborhoods using comma
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [7]:
df1 = df1.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [8]:
df1.loc[df1.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df1[df1.Neighbourhood == 'Not assigned']['Borough']
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


###  Use the .shape method to print the number of rows of your dataframe.

In [9]:
print('Shape of result',df1.shape)

Shape of result (103, 3)
